<a href="https://colab.research.google.com/github/mkane968/Webscraping-Wikipedia-Tables/blob/main/Web_Scraping_Wikipedia_Tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Scrape table values from one Wikipedia page

In [ ]:
#Import libraries
import urllib
from bs4 import BeautifulSoup as bsoup
import bleach
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# get URL
page = requests.get("https://en.wikipedia.org/wiki/Ubisoft_Blue_Byte")
  
# scrape webpage
soup = BeautifulSoup(page.content, 'html.parser')

# display scrapped data (optional)
#print(soup.prettify())

In [ ]:
#Get title of page
title = soup.find(id="firstHeading")
string = title.string
#Turn into string
string = str(string)
#Check title of page
string

'Ubisoft Blue Byte'

In [ ]:
# use the 'find_all' function to bring back all instances of the 'table' 
# tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
#all_tables

In [ ]:
#Find table with needed location data
right_table=soup.find_all('table', class_='infobox vcard')
#right_table

In [ ]:
#Convert table into dataframe
df=pd.read_html(str(right_table))
# Clean datafarme (transpose, add column headers)
df=pd.DataFrame(df[0])
df = df.T
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df

,NaN,Formerly,Type,Industry,Founded,Founders,Headquarters,Number of locations,Key people,Products,Number of employees,Parent,Divisions,Website
1,NaN,Blue Byte (1988–2017),Subsidiary,Video games,"October 1988; 33 years agoMülheim, Germany",Thomas HertzlerLothar Schmitt,"Düsseldorf, Germany",3 studios (2020),Benedikt Grindel (managing director),The SettlersBattle IsleAnno,700[1] (2021),Ubisoft (2001–present),Ubisoft BerlinUbisoft DüsseldorfUbisoft Mainz,bluebyte.ubisoft.com


In [ ]:
#Add desired values to new dataframe (company title and headquarters)
df2 = df.filter(['Headquarters'])
df2['Company'] = [string]

#Clean dataframe
cols = df2.columns.tolist()
cols = cols[-1:] + cols[:-1]
df2 = df2[cols]

#Print dataframe with company and headquarters
df2


,Company,Headquarters
1,Ubisoft Blue Byte,"Düsseldorf, Germany"


#2. Scrape table values from multiple Wikipedia pages 

In [ ]:
#Need to have a list of urls
#Set up lists to append scraped data
soups = []
title_list = []
title_strings = []
data = []
# links from http://en.wikipedia.org/wiki/
link_list = ['/wiki/Ubisoft_Blue_Byte',
    '/wiki/Nintendo',
    '/wiki/Xbox_Game_Studios']

# create function t oharvest data from each URL
def get_data(page_url):
    page = requests.get('https://en.wikipedia.org' + page_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    #Get title of page
    title = soup.find(id="firstHeading")
    title_list.append(title)
    #Append scraped data to list
    soups.append(soup)
   

 # call the function for each URL in the list
for link in link_list:
    get_data(link) 

#Print title of each page to check
for title in title_list: 
  string = title.string
  #Turn into string
  string = str(string)
  title_strings.append(string)
  print(string)  


Ubisoft Blue Byte
Nintendo
Xbox Game Studios


In [ ]:
#Get correct table in each page and append to list
right_tables = []

# use the 'find_all' function to bring back all instances of the 'table' 
for soup in soups: 
  soup.find_all("table")
  #Find table with needed location data
  right_table=soup.find_all('table', class_='infobox vcard')
  right_tables.append(right_table)


In [ ]:
#Append values from each correct table to master dataframe
master_df = pd.DataFrame()

#Convert table into dataframe
for table in right_tables:
  master_df = master_df.append(pd.read_html(str(table)))
  

In [ ]:
# Clean datafarme (transpose, add column headers)
master_df = master_df.T
master_df.columns = master_df.iloc[0]
master_df = master_df.drop(master_df.index[0])
master_df


,NaN,Formerly,Type,Industry,Founded,Founders,Headquarters,Number of locations,Key people,Products,...,Industry,Predecessor,Founded,Headquarters,Area served,Key people,Products,Parent,Subsidiaries,Website
1,NaN,Blue Byte (1988–2017),Subsidiary,Video games,"October 1988; 33 years agoMülheim, Germany",Thomas HertzlerLothar Schmitt,"Düsseldorf, Germany",3 studios (2020),Benedikt Grindel (managing director),The SettlersBattle IsleAnno,...,Video games,Microsoft Games Group,March 2000; 22 years ago,"Redmond, Washington, US",Worldwide,Phil Spencer(CEO of Microsoft Gaming)Matt Boot...,See List of Xbox Game Studios video games,Microsoft,See § Subsidiaries and divisions,xbox.com/xbox-game-studios


In [ ]:
#Keep only headquarters values
master_df2 = master_df['Headquarters']
master_df2

#Transpose and drop index
master_df_t = master_df2.T
master_df_t = master_df_t.reset_index()
master_df_t = master_df_t .drop(master_df_t.columns[[0]],axis = 1)


In [ ]:
#Add titles and rename columns
master_df_t.rename(columns={ master_df_t.columns[0]: "Headquarters" }, inplace = True)
master_df_t['Company'] = title_strings

In [ ]:
#Clean dataframe
cols = master_df_t.columns.tolist()
cols = cols[-1:] + cols[:-1]
master_df_t = master_df_t[cols]

#Print dataframe
master_df_t


,Company,Headquarters
0,Ubisoft Blue Byte,"Düsseldorf, Germany"
1,Nintendo,"11–1 Kamitoba Hokodatecho, Minami-ku, Kyoto, J..."
2,Xbox Game Studios,"Redmond, Washington, US"


In [ ]:
#Export to csv
from google.colab import files
master_df_t.to_csv('headquarters_output.csv', encoding = 'utf-8-sig') 
files.download('headquarters_output.csv')

Sources


*   https://medium.com/analytics-vidhya/web-scraping-a-wikipedia-table-into-a-dataframe-c52617e1f451
*   https://stackoverflow.com/questions/39299658/how-to-scrape-data-from-multiple-wikipedia-pages-with-python

